<a href="https://colab.research.google.com/github/CleanPegasus/Anime_generation_with_DCGAN/blob/master/Anime_Character_Generation_with_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyDrive

     |████████████████████████████████| 993kB 51.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [75]:
!pip install --upgrade scipy

Requirement already up-to-date: scipy in /usr/local/lib/python3.6/dist-packages (1.3.0)


In [0]:
import os 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '0B4wZXrs0DHMHMEl1ODVpMjRTWEk'})
download.GetContentFile('anime-faces.tar.gz')

In [26]:
ls

adc.json  anime-faces/  anime-faces.tar.gz  sample_data/


In [0]:
!tar -xvzf anime-faces.tar.gz

In [1]:
cd anime-faces

/content/anime-faces


In [2]:
ls

 1boy/                green_eyes/          pointy_ears/
 1girl/               green_hair/          ponytail/
 ahoge/               hairband/            purple_eyes/
 akaza_akari/         hair_bow/            purple_hair/
 akemi_homura/        hairclip/            red_background/
 animal_ears/         hair_flower/         red_eyes/
 apron/               hair_ornament/       red_hair/
 aqua_eyes/           hair_ribbon/        'rem_(re:zero)'/
 aqua_hair/           hakurei_reimu/       ribbon/
 armor/               hat/                 santa_costume/
 ayanami_rei/         headphones/          scarf/
 ayase_eli/           honma_meiko/         school_uniform/
 bare_shoulders/      hoshizora_rin/       serafuku/
 black_background/    japanese_clothes/    short_hair/
 black_eyes/          kaname_madoka/       side_ponytail/
 black_hair/          kimono/              silver_hair/
 blazer/              kirisame_marisa/     simple_background/
 blonde_hair/         koizumi_hanayo/      sky/
 blue

In [0]:
!rm -rf yellow_eyes yazawa_nico witch_hat water very_long_hair nishizumi_miho nishikino_maki sonada_umi underwear open_mouth

In [0]:
!rm config.json annotations.csv

In [22]:
!rm -v ._.DS_Store .DS_Store

removed '._.DS_Store'


In [0]:
ls

In [12]:
cd ..

/content


In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import time
import os
from PIL import Image

import tensorflow as tf
from keras import Input, Model
from keras.applications import InceptionResNetV2
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, Flatten, Dense, BatchNormalization, Reshape, concatenate, LeakyReLU, Lambda, Conv2DTranspose, Activation, UpSampling2D, Dropout, ReLU, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical
from keras_preprocessing import image
from keras.models import load_model, Sequential
import keras.backend as K

Using TensorFlow backend.


In [0]:
def load_images(src):
  #src= 'anime-faces'
  image_data = []
  count = 0
  for file in os.listdir(src):
    file_path = os.path.join(src, file)
    for image in os.listdir(file_path):
      if not image.startswith('.'):
        img_path = os.path.join(file_path, image)    
        #print(img_path)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64, 64))
        #print(img.shape)
        img = (img-127.5)/127.5
        image_data.append(img)
        count = count + 1
        #print(len(image_data))
        
        if count > 50000:
          #print("Break")
          break
          
  image_data = np.asarray(image_data)
  #print(image_data.shape)
  #image_data = (image_data - 127.5)/127.5
  return(image_data)
          

In [0]:
def build_gen():
  
  
  gen_model = Sequential()
  gen_model.add(Dense(units = 2048))
  gen_model.add(LeakyReLU(alpha = 0.2))
  
  gen_model.add(Dense(246 * 8 * 8))
  gen_model.add(BatchNormalization())
  gen_model.add(LeakyReLU(alpha = 0.2))
  
  gen_model.add(Reshape((8, 8, 256), input_shape = (256 * 8 * 8,)))
  gen_model.add(UpSampling2D(size = (2, 2)))
  gen_model.add(Conv2D(128, (5, 5), padding = 'same'))
  gen_model.add(LeakyReLU(alpha = 0.2))
  
  gen_model.add(UpSampling2D(size = (2, 2)))
  gen_model.add(Conv2D(64, (5, 5), padding = 'same'))
  gen_model.add(LeakyReLU(alpha = 0.2))
  
  gen_model.add(UpSampling2D(size = (2, 2)))
  gen_model.add(Conv2D(3, (5, 5), padding = 'same'))
  gen_model.add(ReLU())
  
  return gen_model

In [0]:
def build_disc():
  
  disc_model = Sequential()
  
  disc_model.add(Conv2D(128 , (5,5), padding = 'same', input_shape = (96, 96, 3)))
  disc_model.add(LeakyReLU(alpha = 0.2))
  disc_model.add(MaxPooling2D(pool_size = (2, 2)))
  
  disc_model.add(Conv2D(512 , (5,5), padding = 'same', input_shape = (96, 96, 3)))
  disc_model.add(LeakyReLU(alpha = 0.2))
  disc_model.add(MaxPooling2D(pool_size = (2, 2)))
  
  disc_model.add(Flatten())
  
  disc_model.add(Dense(1024))
  disc_model.add(LeakyReLU(alpha = 0.2))
  
  disc_model.add(Dense(1))
  disc_model.add(Activation('sigmoid'))
  
  return disc_model

In [0]:
def save_rgb_img(img, path):

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()

In [0]:
def train():
  
  start_time = time.time()
  epochs = 10000
  batch_size = 128
  z_shape = 100
  lr = 0.0005
  momentum = 0.9
  
  print("Loading Images")
  
  X = load_images('anime-faces')
  print(X.shape)
  
  print("Images Loaded")
  
  #X = (X-127.5)/127.5
  
  print("Normalized Images")
  
  disc_optimizer = SGD(lr = lr, momentum = momentum, nesterov = True)
  gen_optimizer = SGD(lr = lr, momentum = momentum, nesterov = True)
  
  gen_model = build_gen()
  gen_model.compile(loss = 'binary_crossentropy', optimizer = gen_optimizer)
  
  disc_model = build_disc()
  disc_model.compile(loss = 'binary_crossentropy', optimizer = disc_optimizer)
  
  adversarial_model = Sequential()
  adversarial_model.add(gen_model)
  disc_model.trainable = False
  adversarial_model.add(disc_model)
  
  adversarial_model.compile(loss = 'binary_crossentropy', optimizer = gen_optimizer)
  
  #tensorboard = TensorBoard(log_dir = "logs/{}".format(time.time()), write_images = True, write_grads = True , write_graphs = True)
  #tensorboard.set_model(gen_model)
  #tensorboard.set_model(disc_model)
  
  for epoch in range(epochs):
    
    print("Epoch: ", epoch)
    
    number_of_batches = int(X.shape[0] / batch_size)
    
    print("Number of Batches: ", number_of_batches)
    
    for index in range(number_of_batches):
      
      # Training the generator
      
      z_noise = np.random.normal(0, 1, size = (batch_size, z_shape))
      
      image_batch = X[index * batch_size : (index + 1) * batch_size]
      
      generated_images = gen_model.predict_on_batch(z_noise)
      
      y_real = np.ones(batch_size) - np.random.random_sample(batch_size) * 0.2
      
      y_fake = np.random.random_sample(batch_size) * 0.2
      
      disc_loss_real = disc_model.train_on_batch(image_batch, y_real)
      disc_loss_fake = disc_model.train_on_batch(generated_images, y_fake)
      
      disc_loss = np.mean(disc_loss_real, disc_loss_fake)
      print("Discriminator Loss: ", disc_loss)
      
      
      # Training the adversarial
      
      z_noise = np.random.normal(0, 1, size = (batch_size, z_shape))
      
      gen_loss = adversarial_model.train_on_batch(z_noise, [1] * batch_size)
      
      print("Generator Loss: ", gen_loss)
      
      if epoch % 20 == 0:
        
        z_noise = np.random.normal(0, 1, size = (batch_size, z_shape))
        gen_images1 = gen_model.predict_on_batch(z_noise)
        
        for index, img in enumerate(gen_images1[:2]):
          save_rgb_img(img, "results/image_generated_{}_{}.png".format(epoch, index))
          
        gen_model.save("generator.h5")
        disc_model.save("discriminator.h5")
        
        
  print("Time:", (time.time() - start_time))

In [7]:
if __name__ == '__main__':
  train()

Loading Images
(50065, 64, 64, 3)
Images Loaded
Normalized Images
Instructions for updating:
Colocations handled automatically by placer.
Epoch:  0
Number of Batches:  391


ValueError: ignored